In [7]:
# --- STEP 1: Install Dependencies ---
!pip install osmnx geopandas shapely

import osmnx as ox
import geopandas as gpd
import pandas as pd
from google.colab import files
from shapely.geometry import Point
import os

# --- STEP 2: Manual Upload & Shapefile Processing ---
print("Please upload ALL 6 shapefile files (statistical_areas_2022.shp, .dbf, .prj, .shx, .cpg, .xml):")
uploaded_areas = files.upload()

# Load the shapefile from the temporary directory
print("Reading shapefile...")
gdf_all = gpd.read_file('statistical_areas_2022.shp')
print(f"✓ Loaded {len(gdf_all)} national statistical areas.")

# 1. Filter for Eilat (City Code = 2600)
eilat_areas = gdf_all[gdf_all['SEMEL_YISH'] == 2600].copy()

# 2. Convert coordinates to WGS84 (Degrees)
# OSM data is in Lat/Lng degrees; Shapefiles are usually in Meters (Israel TM Grid).
# This conversion is mandatory for the spatial join to work.
if eilat_areas.crs != "EPSG:4326":
    print(f"Converting coordinates from {eilat_areas.crs} to WGS84...")
    eilat_areas = eilat_areas.to_crs("EPSG:4326")

# --- STEP 3: Define OSM Infrastructure Tags ---
# This dictionary defines what specific categories we pull from OpenStreetMap
# --- STEP 4: Define Broad OSM Tags ---
# Setting a category to 'True' pulls EVERY feature in that category (thousands of points).

tags = {
    # 1. Amenities: Education, Health, Religion, Finance, and Public Services
    'amenity': [
        'university', 'college', 'school', 'kindergarten',        # Education
        'hospital', 'clinic', 'pharmacy', 'dentist', 'doctors',   # Health
        'place_of_worship', 'synagogue', 'mosque', 'church',      # Religion
        'bank', 'atm', 'bureau_de_change',                        # Finance
        'police', 'fire_station', 'post_office', 'townhall',      # Public Services
        'library','arts_centre',                                  # Culture
        'bus_station', 'fuel', 'charging_station', 'parking',     # Transport/Infrastructure
        'shelter', 'recycling',                                   # Safety/Utility
        'bar', 'pub'
    ],

    # 2. Wildcards: These pull EVERY feature in the category (thousands of results)
    'leisure': True,   # Includes: park, playground, gym (fitness_centre), stadium, etc.
    'shop': True,      # Includes: supermarket, mall, and all retail stores.
    'office': True,    # Includes: all professional offices and businesses.
    'craft': True,     # Includes: workshops and craftsmen.

    # 3. Transport & Infrastructure specific keys
    'highway': ['bus_stop'],
    'public_transport': ['platform'],
    'railway': ['station'],

    # 4. Specific Safety keys
    'shelter_type': ['public_bomb_shelter']
}
# --- STEP 5: Fetch & Process Data ---
print("Fetching DEEP Eilat infrastructure from OpenStreetMap...")
print("Note: This may take a minute as we are pulling thousands of rows...")
gdf_osm = ox.features_from_place("Eilat, Israel", tags=tags)

# Convert all geometries (polygons/lines) to Points (centroids)
gdf_osm['geometry'] = gdf_osm['geometry'].centroid
gdf_osm['location_lat'] = gdf_osm.geometry.y
gdf_osm['location_lng'] = gdf_osm.geometry.x

# --- HELPER FUNCTION: Prevent crashes if a column is missing ---
def safe_get(df, col_name):
    if col_name in df.columns:
        return df[col_name]
    # Return a Series of empty values (NaN) if the column doesn't exist
    return pd.Series(index=df.index, dtype='object')

# Consolidate type into one unified column safely
gdf_osm['facility_type'] = (
    safe_get(gdf_osm, 'amenity')
    .fillna(safe_get(gdf_osm, 'shop'))
    .fillna(safe_get(gdf_osm, 'leisure'))
    .fillna(safe_get(gdf_osm, 'office'))
    .fillna(safe_get(gdf_osm, 'highway'))
    .fillna(safe_get(gdf_osm, 'railway'))
    .fillna(safe_get(gdf_osm, 'craft'))
    .fillna('other')
)

# Fill missing names with the facility type (so the file isn't empty)
gdf_osm['name'] = gdf_osm['name'].fillna(gdf_osm['facility_type'].astype(str).str.replace('_', ' ').str.title())

# Keep only necessary columns
cols = ['name', 'facility_type', 'location_lat', 'location_lng', 'geometry']
gdf_osm = gdf_osm[[c for c in cols if c in gdf_osm.columns]].copy()

# Keep only necessary columns
cols = ['name', 'facility_type', 'location_lat', 'location_lng', 'geometry']
gdf_osm = gdf_osm[[c for c in cols if c in gdf_osm.columns]].copy()

# --- STEP 6: Spatial Join (Assign Points to Areas) ---
print("Assigning facilities to statistical areas...")
# Check which statistical area polygon contains each facility point
joined = gpd.sjoin(gdf_osm, eilat_areas[['STAT_2022', 'geometry']], how='left', predicate='within')

# --- STEP 7: Final Cleanup & Direct Download ---
# Prepare for Database (rename to lowercase to match your schema)
final_df = joined.drop(columns=['index_right', 'geometry'])
final_df.rename(columns={'STAT_2022': 'stat_2022'}, inplace=True)
final_df['semel_yish'] = 2600 # Static code for Eilat

# Save the final CSV locally in Colab
local_filename = "Eilat_osm_facilities_with_stat.csv"
final_df.to_csv(local_filename, index=False, encoding='utf-8-sig')



# Trigger the browser download
print(f"✓ Success! Found {len(final_df)} facilities.")
print("Starting download...")
#files.download(local_filename)

Please upload ALL 6 shapefile files (statistical_areas_2022.shp, .dbf, .prj, .shx, .cpg, .xml):


Saving statistical_areas_2022.cpg to statistical_areas_2022 (5).cpg
Saving statistical_areas_2022.dbf to statistical_areas_2022 (5).dbf
Saving statistical_areas_2022.prj to statistical_areas_2022 (5).prj
Saving statistical_areas_2022.shp to statistical_areas_2022 (5).shp
Saving statistical_areas_2022.shx to statistical_areas_2022 (5).shx
Saving statistical_areas_2022.xml to statistical_areas_2022 (5).xml
Reading shapefile...
✓ Loaded 25 national statistical areas.
Converting coordinates from EPSG:2039 to WGS84...
Fetching DEEP Eilat infrastructure from OpenStreetMap...
Note: This may take a minute as we are pulling thousands of rows...
Assigning facilities to statistical areas...
✓ Success! Found 1232 facilities.
Starting download...


/tmp/ipython-input-3949131059.py:69: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_osm['geometry'] = gdf_osm['geometry'].centroid


In [9]:
# --- STEP 8: Visualize Statistical Areas Only ---
print("Generating map of Statistical Areas...")

# Create an interactive map of the polygons
# We color by 'STAT_2022' to distinguish different zones
m8 = eilat_areas.explore(
    column='STAT_2022',      # Color polygons by area ID
    cmap='viridis',          # Color palette
    tooltip=['STAT_2022'],   # Show area ID on hover
    style_kwds=dict(fillOpacity=0.4, color="white", weight=1),
    name="Statistical Areas"
)

m8 # Display the map

Generating map of Statistical Areas...


In [11]:
# --- STEP 9: Visualize Areas + All Points ---
print("Generating combined map (Areas + Points)...")

# Ensure the column name is lowercase 'stat_2022' to match the tooltip
if 'STAT_2022' in joined.columns:
    joined = joined.rename(columns={'STAT_2022': 'stat_2022'})

# Convert the joined DataFrame back to a GeoDataFrame for mapping
gdf_points = gpd.GeoDataFrame(
    joined,
    geometry=gpd.points_from_xy(joined.location_lng, joined.location_lat),
    crs="EPSG:4326"
)

# Create the base map with the Areas
m9 = eilat_areas.explore(
    column='STAT_2022',
    cmap='Set3',
    style_kwds=dict(fillOpacity=0.2),
    name="Areas"
)

# Add the points layer on top
gdf_points.explore(
    m=m9,
    color='blue',
    marker_kwds=dict(radius=3),
    tooltip=['name', 'facility_type', 'stat_2022'], # Now lowercase matches
    name="All Facilities"
)

m9 # Display the map

Generating combined map (Areas + Points)...


In [12]:
# --- STEP 10: Identify and Visualize Missing Points ---
print("Checking for points without a statistical area (Outliers)...")

# Points where 'stat_2022' is NaN are outside the statistical area polygons
missing_points = gdf_points[gdf_points['stat_2022'].isna()].copy()

if missing_points.empty:
    print("Success! All points are correctly assigned to statistical areas.")
else:
    print(f"Found {len(missing_points)} points without an assigned area.")

    # Create a base map with low opacity for context
    m10 = eilat_areas.explore(
        style_kwds=dict(fillOpacity=0.1, color="black"),
        name="Base Map"
    )

    # Add the missing points in Red
    missing_points.explore(
        m=m10,
        color='red',
        marker_type='marker',
        popup=True,
        name="Missing Points"
    )

    display(m10) # Display the map

Checking for points without a statistical area (Outliers)...
Found 6 points without an assigned area.
